# Exploring Impact of Self-supervised Learning on Active Learning

In [ ]:
!pip install simplejson
!pip install yacs
!pip install lightly
%cd deep-active-learning-pytorch/tools

In [21]:
!pip install wandb
!wandb login

wandb: Currently logged in as: zhihanhu99. Use `wandb login --relogin` to force relogin


In [2]:
# set random seeds
import random
import numpy as np
import torch


torch.manual_seed(1)
random.seed(1)
np.random.seed(1)

## AL

The following three commands are for training with pure AL strategies.

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train_al.py \
    --cfg=/content/drive/MyDrive/ML4NLP_FINAL/RESNET18.yaml --al=random --exp-name=resnet18-random

Experiment Directory Already Exists: /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/output/CIFAR10/resnet18/resnet18-random. Reusing it may lead to loss of old logs in the directory.


======== PREPARING DATA AND MODEL ========

Preprocess Operations Selected ==>  [RandomCrop(size=(32, 32), padding=4), RandomHorizontalFlip(p=0.5), ToTensor(), Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.2435, 0.2616])]
Files already downloaded and verified
Files already downloaded and verified

Dataset CIFAR10 Loaded Sucessfully.
Total Train Size: 50000 and Total Test Size: 10000

Data Partitioning Complete. 
Labeled Set: 5000, Unlabeled Set: 40000, Validation Set: 5000

wandb: Currently logged in as: zhihanhu99. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/tools/wandb/run-20240201_110856-ls90nx37
wandb: Run `wandb offline` to tur

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train_al.py \
    --cfg=/content/drive/MyDrive/ML4NLP_FINAL/RESNET18.yaml --al=uncertainty --exp-name=resnet18-uncertainty

Experiment Directory Already Exists: /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/output/CIFAR10/resnet18/resnet18-uncertainty. Reusing it may lead to loss of old logs in the directory.


======== PREPARING DATA AND MODEL ========

Preprocess Operations Selected ==>  [RandomCrop(size=(32, 32), padding=4), RandomHorizontalFlip(p=0.5), ToTensor(), Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.2435, 0.2616])]
Files already downloaded and verified
Files already downloaded and verified

Dataset CIFAR10 Loaded Sucessfully.
Total Train Size: 50000 and Total Test Size: 10000

Data Partitioning Complete. 
Labeled Set: 5000, Unlabeled Set: 40000, Validation Set: 5000

wandb: Currently logged in as: zhihanhu99. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/tools/wandb/run-20240201_125447-9lqhg1c8
wandb: Run `wandb offline` t

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train_al.py \
    --cfg=/content/drive/MyDrive/ML4NLP_FINAL/RESNET18.yaml --al=coreset --exp-name=resnet18-coreset

Experiment Directory Already Exists: /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/output/CIFAR10/resnet18/resnet18-coreset. Reusing it may lead to loss of old logs in the directory.


======== PREPARING DATA AND MODEL ========

Preprocess Operations Selected ==>  [RandomCrop(size=(32, 32), padding=4), RandomHorizontalFlip(p=0.5), ToTensor(), Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.2435, 0.2616])]
Files already downloaded and verified
Files already downloaded and verified

Dataset CIFAR10 Loaded Sucessfully.
Total Train Size: 50000 and Total Test Size: 10000

Data Partitioning Complete. 
Labeled Set: 5000, Unlabeled Set: 40000, Validation Set: 5000

wandb: Currently logged in as: zhihanhu99. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/tools/wandb/run-20240201_150139-0dtlng5r
wandb: Run `wandb offline` to tu

## 100SSL

In [4]:
import copy
import random
import numpy as np
import os

import torch
import torchvision
from torch import nn

from lightly.loss import NTXentLoss
from lightly.models.modules import MoCoProjectionHead
from lightly.models.utils import deactivate_requires_grad, update_momentum
from lightly.transforms.moco_transform import MoCoV2Transform
from lightly.utils.scheduler import cosine_schedule

# create MoCo V2 model
class MoCo(nn.Module):
    def __init__(self, backbone):
        super().__init__()

        self.backbone = backbone
        self.projection_head = MoCoProjectionHead(512, 512, 128)

        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)

        deactivate_requires_grad(self.backbone_momentum)
        deactivate_requires_grad(self.projection_head_momentum)

    def forward(self, x):
        query = self.backbone(x).flatten(start_dim=1)
        query = self.projection_head(query)
        return query

    def forward_momentum(self, x):
        key = self.backbone_momentum(x).flatten(start_dim=1)
        key = self.projection_head_momentum(key).detach()
        return key

resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = MoCo(backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

transform = MoCoV2Transform(input_size=32)
dataset = torchvision.datasets.CIFAR10(
    "datasets/cifar10", download=True, transform=transform
)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    shuffle=True,
    drop_last=True,
    num_workers=2,
)

criterion = NTXentLoss(memory_bank_size=4096)
optimizer = torch.optim.SGD(model.parameters(), lr=0.06)

def train_moco(epochs):
    "Set up a simple training loop for MoCo"
    for epoch in range(epochs):
        total_loss = 0
        momentum_val = cosine_schedule(epoch, epochs, 0.996, 1)
        for batch in dataloader:
            x_query, x_key = batch[0]
            update_momentum(model.backbone, model.backbone_momentum, m=momentum_val)
            update_momentum(
                model.projection_head, model.projection_head_momentum, m=momentum_val
            )
            x_query = x_query.to(device)
            x_key = x_key.to(device)
            query = model(x_query)
            key = model.forward_momentum(x_key)
            loss = criterion(query, key)
            total_loss += loss.detach()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        avg_loss = total_loss / len(dataloader)
        print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

100%|██████████| 170498071/170498071 [00:13<00:00, 12710983.88it/s]


Extracting datasets/cifar10/cifar-10-python.tar.gz to datasets/cifar10


/usr/local/lib/python3.10/dist-packages/lightly/models/modules/memory_bank.py:88: UserWarning: Memory bank size 'size=4096' does not specify feature dimension. It is recommended to set the feature dimension with 'size=(n, dim)' when creating the memory bank. Distributed training might fail if the feature dimension is not set.
  warnings.warn(


In [ ]:
epochs = 100

print("Starting Training")
train_moco(epochs)

# save the model checkpoint
os.makedirs("lightly", exist_ok=True)
PATH = "lightly/moco_100epoch.tar"

torch.save({
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
    }, PATH)

Files already downloaded and verified
Starting Training


/usr/local/lib/python3.10/dist-packages/lightly/models/modules/memory_bank.py:88: UserWarning: Memory bank size 'size=4096' does not specify feature dimension. It is recommended to set the feature dimension with 'size=(n, dim)' when creating the memory bank. Distributed training might fail if the feature dimension is not set.
  warnings.warn(


epoch: 00, loss: 8.08506
epoch: 01, loss: 7.76627
epoch: 02, loss: 7.57730
epoch: 03, loss: 7.48487
epoch: 04, loss: 7.42898
epoch: 05, loss: 7.39658
epoch: 06, loss: 7.36479
epoch: 07, loss: 7.33997
epoch: 08, loss: 7.32268
epoch: 09, loss: 7.30586
epoch: 10, loss: 7.28873
epoch: 11, loss: 7.27858
epoch: 12, loss: 7.26322
epoch: 13, loss: 7.25206
epoch: 14, loss: 7.24331
epoch: 15, loss: 7.23122
epoch: 16, loss: 7.22231
epoch: 17, loss: 7.22205
epoch: 18, loss: 7.21295
epoch: 19, loss: 7.20859
epoch: 20, loss: 7.19597
epoch: 21, loss: 7.19485
epoch: 22, loss: 7.18807
epoch: 23, loss: 7.18351
epoch: 24, loss: 7.17850
epoch: 25, loss: 7.17488
epoch: 26, loss: 7.16929
epoch: 27, loss: 7.16588
epoch: 28, loss: 7.16558
epoch: 29, loss: 7.15743
epoch: 30, loss: 7.15486
epoch: 31, loss: 7.15097
epoch: 32, loss: 7.14735
epoch: 33, loss: 7.14601
epoch: 34, loss: 7.14329
epoch: 35, loss: 7.13994
epoch: 36, loss: 7.13548
epoch: 37, loss: 7.13443
epoch: 38, loss: 7.12989
epoch: 39, loss: 7.12820


In [ ]:
# The MoCo model has different naming for the layers than the downstream AL model
# We need to map the layers from the MoCo model to the AL model

md_moco = torch.load("lightly/moco_100epoch.tar")

# put the path of one of the best performing models here
md_al = torch.load("deep-active-learning-pytorch/output/CIFAR10/resnet18/resnet18-coreset/episode_2/vlBest_acc_85_model_epoch_0164.pyth")

md_lst = [k for k, _ in md_moco["model_state"].items()]
md_al_lst = [k for k, _ in md_al["model_state"].items()]

md_lst = md_lst[md_lst.index("backbone.4.0.conv1.weight"):md_lst.index("backbone.7.1.bn2.num_batches_tracked")+1]
md_al_lst = md_al_lst[md_al_lst.index("layer1.0.conv1.weight"):md_al_lst.index("layer4.1.bn2.num_batches_tracked")+1]

new_state = {}
for i, k in enumerate(md_lst):
  new_state[md_al_lst[i]] = md_moco["model_state"][k]

md_moco["model_state"] = new_state

# save the new model checkpoint
PATH = "/content/drive/MyDrive/ML4NLP_FINAL/lightly/moco_100epoch_new.tar"

torch.save({
        "epoch": 100,
        "model_state": md_moco["model_state"],
        "optimizer_state": optimizer.state_dict(),
    }, PATH)

The following commands are for AL training after 100 epochs of SSL.

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train_al.py \
    --cfg=/content/drive/MyDrive/ML4NLP_FINAL/RESNET18_100SSL.yaml --al=random --exp-name=ssl-100-random

Experiment Directory Already Exists: /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/output/CIFAR10/resnet18/ssl-100-random. Reusing it may lead to loss of old logs in the directory.


======== PREPARING DATA AND MODEL ========

Preprocess Operations Selected ==>  [RandomCrop(size=(32, 32), padding=4), RandomHorizontalFlip(p=0.5), ToTensor(), Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.2435, 0.2616])]
Files already downloaded and verified
Files already downloaded and verified

Dataset CIFAR10 Loaded Sucessfully.
Total Train Size: 50000 and Total Test Size: 10000

Data Partitioning Complete. 
Labeled Set: 5000, Unlabeled Set: 40000, Validation Set: 5000

wandb: Currently logged in as: zhihanhu99. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/tools/wandb/run-20240131_145113-dc5jwe4g
wandb: Run `wandb offline` to turn

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train_al.py \
    --cfg=/content/drive/MyDrive/ML4NLP_FINAL/RESNET18_100SSL.yaml --al=uncertainty --exp-name=ssl-100-uncertainty

Experiment Directory Already Exists: /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/output/CIFAR10/resnet18/ssl-100-uncertainty. Reusing it may lead to loss of old logs in the directory.


======== PREPARING DATA AND MODEL ========

Preprocess Operations Selected ==>  [RandomCrop(size=(32, 32), padding=4), RandomHorizontalFlip(p=0.5), ToTensor(), Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.2435, 0.2616])]
Files already downloaded and verified
Files already downloaded and verified

Dataset CIFAR10 Loaded Sucessfully.
Total Train Size: 50000 and Total Test Size: 10000

Data Partitioning Complete. 
Labeled Set: 5000, Unlabeled Set: 40000, Validation Set: 5000

wandb: Currently logged in as: zhihanhu99. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/tools/wandb/run-20240201_052236-i8gx8rnl
wandb: Run `wandb offline` to

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train_al.py \
    --cfg=/content/drive/MyDrive/ML4NLP_FINAL/RESNET18_100SSL.yaml --al=coreset --exp-name=ssl-100-coreset

Experiment Directory Already Exists: /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/output/CIFAR10/resnet18/ssl-100-coreset. Reusing it may lead to loss of old logs in the directory.


======== PREPARING DATA AND MODEL ========

Preprocess Operations Selected ==>  [RandomCrop(size=(32, 32), padding=4), RandomHorizontalFlip(p=0.5), ToTensor(), Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.2435, 0.2616])]
Files already downloaded and verified
Files already downloaded and verified

Dataset CIFAR10 Loaded Sucessfully.
Total Train Size: 50000 and Total Test Size: 10000

Data Partitioning Complete. 
Labeled Set: 5000, Unlabeled Set: 40000, Validation Set: 5000

wandb: Currently logged in as: zhihanhu99. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/tools/wandb/run-20240201_092941-zjml03nv
wandb: Run `wandb offline` to tur

## 500SSL

In [ ]:
import copy
import random
import numpy as np
import os

import torch
import torchvision
from torch import nn

from lightly.loss import NTXentLoss
from lightly.models.modules import MoCoProjectionHead
from lightly.models.utils import deactivate_requires_grad, update_momentum
from lightly.transforms.moco_transform import MoCoV2Transform
from lightly.utils.scheduler import cosine_schedule

# create MoCo V2 model
class MoCo(nn.Module):
    def __init__(self, backbone):
        super().__init__()

        self.backbone = backbone
        self.projection_head = MoCoProjectionHead(512, 512, 128)

        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)

        deactivate_requires_grad(self.backbone_momentum)
        deactivate_requires_grad(self.projection_head_momentum)

    def forward(self, x):
        query = self.backbone(x).flatten(start_dim=1)
        query = self.projection_head(query)
        return query

    def forward_momentum(self, x):
        key = self.backbone_momentum(x).flatten(start_dim=1)
        key = self.projection_head_momentum(key).detach()
        return key

resnet = torchvision.models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])
model = MoCo(backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

transform = MoCoV2Transform(input_size=32)
dataset = torchvision.datasets.CIFAR10(
    "datasets/cifar10", download=True, transform=transform
)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    shuffle=True,
    drop_last=True,
    num_workers=2,
)

criterion = NTXentLoss(memory_bank_size=4096)
optimizer = torch.optim.SGD(model.parameters(), lr=0.06)

def train_moco(epochs):
    "Set up a simple training loop for MoCo"
    for epoch in range(epochs):
        total_loss = 0
        momentum_val = cosine_schedule(epoch, epochs, 0.996, 1)
        for batch in dataloader:
            x_query, x_key = batch[0]
            update_momentum(model.backbone, model.backbone_momentum, m=momentum_val)
            update_momentum(
                model.projection_head, model.projection_head_momentum, m=momentum_val
            )
            x_query = x_query.to(device)
            x_key = x_key.to(device)
            query = model(x_query)
            key = model.forward_momentum(x_key)
            loss = criterion(query, key)
            total_loss += loss.detach()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        avg_loss = total_loss / len(dataloader)
        print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

In [5]:
epochs = 500

print("Starting Training")
train_moco(epochs)

# save the model checkpoint
PATH = "lightly/moco_500epoch.tar"

torch.save({
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
    }, PATH)

Starting Training
epoch: 00, loss: 8.10422
epoch: 01, loss: 7.79000
epoch: 02, loss: 7.59594
epoch: 03, loss: 7.50217
epoch: 04, loss: 7.44728
epoch: 05, loss: 7.41411
epoch: 06, loss: 7.38246
epoch: 07, loss: 7.35500
epoch: 08, loss: 7.33807
epoch: 09, loss: 7.32140
epoch: 10, loss: 7.30748
epoch: 11, loss: 7.29639
epoch: 12, loss: 7.28387
epoch: 13, loss: 7.27345
epoch: 14, loss: 7.26473
epoch: 15, loss: 7.25112
epoch: 16, loss: 7.24300
epoch: 17, loss: 7.24086
epoch: 18, loss: 7.23093
epoch: 19, loss: 7.22600
epoch: 20, loss: 7.21188
epoch: 21, loss: 7.20994
epoch: 22, loss: 7.20390
epoch: 23, loss: 7.19838
epoch: 24, loss: 7.19174
epoch: 25, loss: 7.18609
epoch: 26, loss: 7.18081
epoch: 27, loss: 7.17549
epoch: 28, loss: 7.17592
epoch: 29, loss: 7.16704
epoch: 30, loss: 7.16199
epoch: 31, loss: 7.15738
epoch: 32, loss: 7.15394
epoch: 33, loss: 7.15170
epoch: 34, loss: 7.14762
epoch: 35, loss: 7.14496
epoch: 36, loss: 7.14119
epoch: 37, loss: 7.13766
epoch: 38, loss: 7.13274
epoch: 

In [ ]:
# The MoCo model has different naming for the layers than the downstream AL model
# We need to map the layers from the MoCo model to the AL model

md_moco = torch.load("lightly/moco_500epoch.tar")

# put the path of one of the best performing models here
md_al = torch.load("deep-active-learning-pytorch/output/CIFAR10/resnet18/resnet18-coreset/episode_2/vlBest_acc_85_model_epoch_0164.pyth")

md_lst = [k for k, v in md_moco["model_state"].items()]
md_al_lst = [k for k, v in md_al["model_state"].items()]

md_lst = md_lst[md_lst.index("backbone.4.0.conv1.weight"):md_lst.index("backbone.7.1.bn2.num_batches_tracked")+1]
md_al_lst = md_al_lst[md_al_lst.index("layer1.0.conv1.weight"):md_al_lst.index("layer4.1.bn2.num_batches_tracked")+1]

new_state = {}
for i, k in enumerate(md_lst):
  new_state[md_al_lst[i]] = md_moco["model_state"][k]

md_moco["model_state"] = new_state

# save the new model checkpoint
PATH = "/content/drive/MyDrive/ML4NLP_FINAL/lightly/moco_500epoch_new.tar"

torch.save({
        "epoch": 100,
        "model_state": md_moco["model_state"],
        "optimizer_state": optimizer.state_dict(),
    }, PATH)

The following commands are for AL training after 500 epochs of SSL.

In [25]:
!CUDA_VISIBLE_DEVICES=0 python train_al.py \
    --cfg=/content/drive/MyDrive/ML4NLP_FINAL/RESNET18_500SSL.yaml --al=random --exp-name=ssl-500-random

Experiment Directory is /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/output/CIFAR10/resnet18/ssl-500-random.


======== PREPARING DATA AND MODEL ========

Preprocess Operations Selected ==>  [RandomCrop(size=(32, 32), padding=4), RandomHorizontalFlip(p=0.5), ToTensor(), Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.2435, 0.2616])]
Files already downloaded and verified
Files already downloaded and verified

Dataset CIFAR10 Loaded Sucessfully.
Total Train Size: 50000 and Total Test Size: 10000

Data Partitioning Complete. 
Labeled Set: 5000, Unlabeled Set: 40000, Validation Set: 5000

wandb: Currently logged in as: zhihanhu99. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/tools/wandb/run-20240202_153506-bhs52wim
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run crimson-night-9
wandb: ⭐️ View project

In [26]:
!CUDA_VISIBLE_DEVICES=0 python train_al.py \
    --cfg=/content/drive/MyDrive/ML4NLP_FINAL/RESNET18_500SSL.yaml --al=uncertainty --exp-name=ssl-500-uncertainty

Experiment Directory is /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/output/CIFAR10/resnet18/ssl-500-uncertainty.


======== PREPARING DATA AND MODEL ========

Preprocess Operations Selected ==>  [RandomCrop(size=(32, 32), padding=4), RandomHorizontalFlip(p=0.5), ToTensor(), Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.2435, 0.2616])]
Files already downloaded and verified
Files already downloaded and verified

Dataset CIFAR10 Loaded Sucessfully.
Total Train Size: 50000 and Total Test Size: 10000

Data Partitioning Complete. 
Labeled Set: 5000, Unlabeled Set: 40000, Validation Set: 5000

wandb: Currently logged in as: zhihanhu99. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/tools/wandb/run-20240202_164953-sdhamnpy
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run young-forest-10
wandb: ⭐️ View pr

In [27]:
!CUDA_VISIBLE_DEVICES=0 python train_al.py \
    --cfg=/content/drive/MyDrive/ML4NLP_FINAL/RESNET18_500SSL.yaml --al=coreset --exp-name=ssl-500-coreset

Experiment Directory is /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/output/CIFAR10/resnet18/ssl-500-coreset.


======== PREPARING DATA AND MODEL ========

Preprocess Operations Selected ==>  [RandomCrop(size=(32, 32), padding=4), RandomHorizontalFlip(p=0.5), ToTensor(), Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.2435, 0.2616])]
Files already downloaded and verified
Files already downloaded and verified

Dataset CIFAR10 Loaded Sucessfully.
Total Train Size: 50000 and Total Test Size: 10000

Data Partitioning Complete. 
Labeled Set: 5000, Unlabeled Set: 40000, Validation Set: 5000

wandb: Currently logged in as: zhihanhu99. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /content/drive/MyDrive/ML4NLP_FINAL/deep-active-learning-pytorch/tools/wandb/run-20240202_180513-eo1upvr4
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run distinctive-mountain-11
wandb: ⭐️ Vie